In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split
SEED = 777

In [6]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [7]:
from sklearn.preprocessing import LabelEncoder
features_target = ['target'] 

for feature in features_target:
    le = LabelEncoder()
    le.fit(train_df[feature])
    train_df[feature] = le.transform(train_df[feature])
    
print(train_df['target'].head())

0    5
1    5
2    1
3    7
4    1
Name: target, dtype: int64


In [8]:
train_no_predict = train_df.drop(['id', 'target'], 1)
train_predict = train_df['target']

train_percent = 0.80

X_train, X_test, y_train, y_test = train_test_split(train_no_predict, train_predict, stratify=train_predict, 
                                                    train_size=train_percent, random_state=SEED)
print('train count: ', len(y_train))
print('test count: ', len(y_test))

train count:  160000
test count:  40000


In [13]:
count = len(train_no_predict.columns) + 1
neighbors = [5, 25, 50, 75]
#neighbors = [5, 10]
print(neighbors)

[5, 25, 50, 75]


In [17]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.metrics import log_loss
import datetime
import time


cv_scores = []
test_scores = []
for K in neighbors:
    print('starting KNN', K)
    print(datetime.datetime.now())
    start = time.time()
    
    pca = PCA(n_components=K, random_state=SEED)
    pca.fit(X_train, y_train)
    tX_train = pca.transform(X_train)
    print(tX_train[:1])
    
    knn = KNeighborsClassifier(n_neighbors = K)
    scores = cross_val_score(knn, tX_train, y_train, cv = 2, scoring ="neg_log_loss")
    cv_scores.append(scores.mean())
    print('train neg_log_loss', scores.mean())
    
    #tX_test = pca.transform(X_test)
    #pred = knn.predict_proba(tX_test)
    #test_log_loss = log_loss(y_test, pred)
    #test_scores.append(test_log_loss)
    #print('test neg_log_loss', test_log_loss)
    
    print(datetime.datetime.now())
    minutes = (time.time() - start) / 60
    print('minutes', minutes, '\n')

starting KNN 5
2021-06-26 20:31:30.760703
[[ 20.77121925  -5.2807626   -3.80161556 -11.83341809 -16.24459964]
 [ 44.97675146  37.83858997  -1.9875169   -7.47453568 -10.39881441]
 [  8.13201278 -12.13346852   8.8130465   -2.36726219  -7.2043294 ]
 [ -1.70018823   6.65601288   0.88911081   0.14402263  -0.37799928]
 [-14.40034247   3.96638516  -1.61878401   0.92555457   1.57333269]]
train neg_log_loss -14.092944107480761
2021-06-26 20:31:33.816051
minutes 0.05092179775238037 

starting KNN 25
2021-06-26 20:31:33.816710
[[ 2.07712050e+01 -5.28051476e+00 -3.79856245e+00 -1.18440338e+01
  -1.63419279e+01  4.24722182e+00 -2.19002011e+00  3.32813345e+00
  -1.75350576e+01 -1.83302368e+01  3.77953064e+01  2.10384874e+01
  -1.42988804e+01 -3.90056571e+01 -2.23944282e+01 -2.58446884e+01
   5.78805423e+00 -9.34928940e+00  1.33317758e+01  4.27000542e+00
  -3.43164617e+00 -3.64045655e+00 -2.35163154e+00 -1.89256821e+00
  -3.95603983e+00]
 [ 4.49767574e+01  3.78385506e+01 -1.98765491e+00 -7.48057106e+

train neg_log_loss -2.2451228567788344
2021-06-26 20:42:51.838871
minutes 3.852350334326426 



In [18]:
print(cv_scores)

[-14.092944107480761, -4.092611878857934, -2.6572420402298893, -2.2451228567788344]
